In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
import pandas as pd

# Load the CSV file
file_path = "/content/drive/MyDrive/tesco-urls/final/cleaned_file13_vegan.csv"
df = pd.read_csv(file_path)

In [ ]:
# Define a function to clean the "Unit" column
def clean_unit(value):
    if pd.isna(value):  # Check for NaN
        return value
    value = str(value).lower().replace(" ", "")
    if "100g" in value:
        return "100g"
    elif "100ml" in value:
        return "100ml"
    return value

In [ ]:
import re

# Define a function to clean the "Protein" column
def clean_protein(value):
    if pd.isna(value) or str(value).strip() == "":  # Check for NaN or blank values
        return 0
    # Use a regular expression to extract the numeric part
    match = re.search(r"[\d.]+", str(value))
    return float(match.group()) if match else 0

In [ ]:
# Define a function to clean the "final_unit_price" column
def clean_unit_price(value):
    if pd.isna(value) or str(value).strip() == "":  # Check for NaN or blank values
        return 0
    # Use a regular expression to extract the numeric part
    match = re.search(r"[\d.]+", str(value))
    return float(match.group()) if match else 0

In [ ]:
# Define a function to clean the "final_fat" column
def clean_fat(value):
    if str(value) == "(28.7g)":
      return 28.7
    if pd.isna(value) or str(value).strip() == "":  # Check for NaN or blank values
        return 0
    # Use a regular expression to extract the numeric part
    match = re.search(r"[\d.]+", str(value))
    return float(match.group()) if match else 0

In [ ]:
# Define a function to clean the Energy column
def clean_energy(value):
    if pd.isna(value) or str(value).strip() == "":  # Check for NaN or blank values
        return 0
    # Use a regular expression to extract the numeric part
    low = 0
    high = len(str(value))
    s = str(value).lower()
    if ("kj" in s):
      high = s.index("kj")
      if ("kcal" in s):
        low = s.index("kcal")
    elif ("," in s):
      high = s.index(",")
    if (low > high):
      low = 0
    match = re.search(r"[\d.]+", s[low:high])
    return float(match.group()) if match else 0

In [ ]:
df["Unit"] = df["Unit"].apply(clean_unit)

In [ ]:
df = df[df['final_energy'].notna()]  # Remove NaN values
df = df[df['final_energy'].str.strip() != ""]  # Remove rows with only whitespace

In [ ]:
df["final_protein"] = df["final_protein"].apply(clean_protein)

In [ ]:
df["final_fat"] = df["final_fat"].apply(clean_protein)

In [ ]:
df["final_energy"] = df["final_energy"].apply(clean_energy)

In [ ]:
df = df[df["price_per_kg"].str.strip() != "Price per kg not found"]
df["price_per_kg"] = df["price_per_kg"].apply(clean_unit_price)

In [ ]:
allowed_values = ["100ml", "100g", "100", "/100"]
df = df[df["Unit"].isin(allowed_values)]

In [ ]:
df["Unit"] = df["Unit"].replace(["/100", "100"], "100g")

In [ ]:
df = df[df["price_per_kg"].str.contains("kg|liter", case=False, na=False)]

In [ ]:
urls_to_remove = [
    "https://www.tesco.ie/groceries/en-IE/products/300808531",
    "https://www.tesco.ie/groceries/en-IE/products/264129006",
    "https://www.tesco.ie/groceries/en-IE/products/262489469",
    "https://www.tesco.ie/groceries/en-IE/products/255605263",
    "https://www.tesco.ie/groceries/en-IE/products/314579059",
    "https://www.tesco.ie/groceries/en-IE/products/267073786",
    "https://www.tesco.ie/groceries/en-IE/products/310134264",
    "https://www.tesco.ie/groceries/en-IE/products/265915658"
]
df = df[~df["url"].isin(urls_to_remove)]

In [ ]:
import json

with open("/content/drive/MyDrive/tesco-urls/tesco-vegan-urls.json", "r") as file:
  vegan_urls = json.load(file)

In [ ]:
vegan_urls = set(vegan_urls)
df["vegan"] = df["url"].apply(lambda x: x in vegan_urls)

In [49]:
# removing abnormal row
urls_to_remove = [
    "https://www.tesco.ie/groceries/en-IE/products/300456105"
]
df = df[~df["url"].isin(urls_to_remove)]

In [50]:
df.to_csv("/content/drive/MyDrive/tesco-urls/final/cleaned_file14_vegan.csv", index=False)